**I trained the SmolLM2-135M model to answer data science specific questions using soufyane/DATA_SCIENCE_QA dataset from hugging face.**

In [ ]:
!pip install transformers datasets trl huggingface_hub

In [2]:

# Authenticate to Hugging Face
from huggingface_hub import login

login()
# for convenience you can create an environment variable containing your hub token as HF_TOKEN

In [3]:
# Import necessary libraries

from transformers import AutoModelForCausalLM, AutoTokenizer

from datasets import load_dataset

from trl import SFTConfig, SFTTrainer, setup_chat_format

import torch



device = (

    "cuda"

    if torch.cuda.is_available()

    else "mps" if torch.backends.mps.is_available() else "cpu"

)



# Load the model and tokenizer

model_name = "HuggingFaceTB/SmolLM2-135M"

model = AutoModelForCausalLM.from_pretrained(

    pretrained_model_name_or_path=model_name

).to(device)

tokenizer = AutoTokenizer.from_pretrained(pretrained_model_name_or_path=model_name)



# Set up the chat format

model, tokenizer = setup_chat_format(model=model, tokenizer=tokenizer)



# Set our name for the finetune to be saved &/ uploaded to

finetune_name = "SmolLM2-SFT-DSQA"

finetune_tags = ["smol-lm2-135M", "datascience-qa"]

config.json:   0%|          | 0.00/704 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/269M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/3.66k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/801k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/466k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.10M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/831 [00:00<?, ?B/s]

# Generate with the base model



Here we will try out the base model which does not have a chat template. 

In [4]:
# Let's test the base model before training

prompt = "Explain a general data science project lifecycle"


messages = [{"role": "user", "content": prompt}]

formatted_prompt = tokenizer.apply_chat_template(messages, tokenize=False)



# Generate response

inputs = tokenizer(formatted_prompt, return_tensors="pt").to(device)

outputs = model.generate(**inputs, max_new_tokens=100)

print("Before training:")

print(tokenizer.decode(outputs[0], skip_special_tokens=True))

Before training:
user
Explain a general data science project lifecycle
Explain a general data science project lifecycle
Explain a general data science project lifecycle
Explain a general data science project lifecycle
Explain a general data science project lifecycle
Explain a general data science project lifecycle
Explain a general data science project lifecycle
Explain a general data science project lifecycle
Explain a general data science project lifecycle
Explain a general data science project lifecycle
Explain a general data science project lifecycle
Explain a general data science project lifecycle
Explain a general data science project lifecycle
Explain a general data


## Dataset Preparation



We will load a sample dataset and format it for training. The dataset should be structured with input-output pairs, where each input is a prompt and the output is the expected response from the model.



**TRL will format input messages based on the model's chat templates.** They need to be represented as a list of dictionaries with the keys: `role` and `content`,.

In [5]:
# Load a sample dataset

from datasets import load_dataset


ds = load_dataset(path="soufyane/DATA_SCIENCE_QA")

data (1).csv:   0%|          | 0.00/391k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/1070 [00:00<?, ? examples/s]

In [6]:
# TODO: 🦁 If your dataset is not in a format that TRL can convert to the chat template, you will need to process it. Refer to the [module](../chat_templates.md)
def process_dataset(sample):

    # use the tokenizer's method to apply the chat template
    message_formatted = [
        {"role": 'user', "content": sample['Question']},
        {"role": 'assistant', "content": sample['Answer']},
    ]
    sample['messages'] = message_formatted
    return sample

ds = ds.map(process_dataset)

Map:   0%|          | 0/1070 [00:00<?, ? examples/s]

In [7]:
ds = ds['train'].train_test_split(test_size=0.3)

In [8]:
ds['train'][0]

{'Unnamed: 0': 525,
 'Question': 'Summarize the key idea of survival analysis briefly.',
 'Answer': 'Survival analysis involves studying the duration until specific events happen, such as death or failure, and modeling the associated risks or probabilities over time. It accounts for censoring, where events are not observed for all subjects, and employs techniques like Kaplan-Meier estimation and Cox proportional hazards regression to analyze event times and factors influencing event occurrence.',
 'messages': [{'content': 'Summarize the key idea of survival analysis briefly.',
   'role': 'user'},
  {'content': 'Survival analysis involves studying the duration until specific events happen, such as death or failure, and modeling the associated risks or probabilities over time. It accounts for censoring, where events are not observed for all subjects, and employs techniques like Kaplan-Meier estimation and Cox proportional hazards regression to analyze event times and factors influencing 

## Configuring the SFTTrainer



The `SFTTrainer` is configured with various parameters that control the training process. These include the number of training steps, batch size, learning rate, and evaluation strategy. Adjust these parameters based on your specific requirements and computational resources.

In [9]:
# Configure the SFTTrainer

sft_config = SFTConfig(

    output_dir="./sft_output",

    # 500 would have worked for this one
    max_steps=1000,  # Adjust based on dataset size and desired training duration

    per_device_train_batch_size=4,  # Set according to your GPU memory capacity

    learning_rate=5e-5,  # Common starting point for fine-tuning

    logging_steps=10,  # Frequency of logging training metrics

    save_steps=100,  # Frequency of saving model checkpoints

    evaluation_strategy="steps",  # Evaluate the model at regular intervals

    eval_steps=50,  # Frequency of evaluation

    use_mps_device=(

        True if device == "mps" else False

    ),  # Use MPS for mixed precision training

    hub_model_id=finetune_name,  # Set a unique name for your model

)



# Initialize the SFTTrainer

trainer = SFTTrainer(

    model=model,

    args=sft_config,

    train_dataset=ds["train"],

    tokenizer=tokenizer,

    eval_dataset=ds["test"],

)



# TODO: 🦁 🐕 align the SFTTrainer params with your chosen dataset. For example, if you are using the `bigcode/the-stack-smol` dataset, you will need to choose the `content` column`

/opt/conda/lib/python3.10/site-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/tmp/ipykernel_23/3767605764.py:35: FutureWarning: `tokenizer` is deprecated and removed starting from version 0.16.0 for `SFTTrainer.__init__`. Use `processing_class` instead.
  trainer = SFTTrainer(


Map:   0%|          | 0/749 [00:00<?, ? examples/s]

Map:   0%|          | 0/321 [00:00<?, ? examples/s]

max_steps is given, it will override any value given in num_train_epochs


## Training the Model



With the trainer configured, we can now proceed to train the model. The training process will involve iterating over the dataset, computing the loss, and updating the model's parameters to minimize this loss.

In [10]:
# Train the model

trainer.train()

# Save the model
trainer.save_model(f"./{finetune_name}")

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Step,Training Loss,Validation Loss
50,2.030900,1.918021
100,1.760900,1.869897
150,1.672700,1.843704
200,1.391400,1.840126
250,1.382000,1.864417
300,1.351500,1.864695
350,1.388100,1.855685
400,1.057800,1.917605
450,1.031000,1.944387
500,1.021100,1.941496


In [11]:
trainer.push_to_hub(tags=finetune_tags)

Upload 3 LFS files:   0%|          | 0/3 [00:00<?, ?it/s]

events.out.tfevents.1734466426.e97a5d704eae.23.0:   0%|          | 0.00/32.4k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/538M [00:00<?, ?B/s]

training_args.bin:   0%|          | 0.00/5.56k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/Avibhi/SmolLM2-SFT-DSQA/commit/732fdaa1ad0b45b449946c99ee75dce7ee1080b5', commit_message='End of training', commit_description='', oid='732fdaa1ad0b45b449946c99ee75dce7ee1080b5', pr_url=None, repo_url=RepoUrl('https://huggingface.co/Avibhi/SmolLM2-SFT-DSQA', endpoint='https://huggingface.co', repo_type='model', repo_id='Avibhi/SmolLM2-SFT-DSQA'), pr_revision=None, pr_num=None)

<div style='background-color: lightblue; padding: 10px; border-radius: 5px; margin-bottom: 20px; color:black'>

    <h2 style='margin: 0;color:blue'>Bonus Exercise: Generate with fine-tuned model</h2>

    <p>🐕 Use the fine-tuned to model generate a response, just like with the base example..</p>

</div>

In [14]:
# Test the fine-tuned model on the same prompt


prompt = "Explain a general data science project lifecycle"
messages = [{"role": "user", "content": prompt}]
formatted_prompt = tokenizer.apply_chat_template(messages, tokenize=False)


# Generate response

inputs = tokenizer(formatted_prompt, return_tensors="pt").to(device)


# finetuned model 
model_name = 'Avibhi/SmolLM2-SFT-DSQA'

model = AutoModelForCausalLM.from_pretrained(

    pretrained_model_name_or_path=model_name

).to(device)

outputs = model.generate(**inputs, max_new_tokens=100)

print("After training:")

print(tokenizer.decode(outputs[0], skip_special_tokens=True))

config.json:   0%|          | 0.00/812 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/538M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/132 [00:00<?, ?B/s]

After training:
user
Explain a general data science project lifecycle
assistant
A general data science project lifecycle includes stages from defining business objectives to monitoring and managing data. Initially, business needs and objectives need to be defined. Data collection and organization are established. Data preparation includes cleaning and transforming data, and preparing it for analysis. Data storage and preservation are also considered. Data analysis begins with data analysis tools and techniques. Data interpretation and decision-making are carried out using analytical tools and models. Data management and storage are finalized. Data utilization and stewardship is completed


In [17]:
# Few other examples
prompt = "Explain EDA in 3 sentences"

# Format with template

messages = [{"role": "user", "content": prompt}]

formatted_prompt = tokenizer.apply_chat_template(messages, tokenize=False)

# Generate response

inputs = tokenizer(formatted_prompt, return_tensors="pt").to(device)

outputs = model.generate(**inputs, max_new_tokens=200)

print(tokenizer.decode(outputs[0], skip_special_tokens=True))

user
Explain EDA in 3 sentences
assistant
EDA, or data analysis, is the process of looking at data to understand its meaning and how it fits together. It involves tasks like data cleaning, data visualization, and data analysis to extract insights and make informed decisions from the data. EDA is essential for ensuring data quality, ensuring data is available for analysis, and making informed decisions based on the data. It's like being a detective who solves data problems!

EDA is crucial in various industries, including finance, marketing, and government, to extract valuable insights from data and make decisions that affect businesses and lives. By analyzing data, EDA helps businesses achieve their goals and objectives by understanding data insights and making strategic decisions. Without EDA, data analysis would be a guessing game, and businesses would struggle to make sense of the data they're given.

EDA is a critical component of data management and analysis, contributing to the d

## 💐 You're done!



This notebook provided a step-by-step guide to fine-tuning the `HuggingFaceTB/SmolLM2-135M` model using the `SFTTrainer`. By following these steps, you can adapt the model to perform specific tasks more effectively. If you want to carry on working on this course, here are steps you could try out:



- Try this notebook on a harder difficulty

- Review a colleagues PR

- Improve the course material via an Issue or PR.